In [1]:
%load_ext autoreload
%autoreload 2

# Ligand and Pocket

### Needs rdkit>=2020.03.3

In [2]:
import rdkit
assert '2020.03.3' <= rdkit.__version__

from rdkit import Chem
from cbiprep.ligand_expo import LigandExpo
from cbiprep.pdbatoms import PDBAtoms
from cbiprep.matrix import GetTopologicalMatrix
from cbiprep.atomtyper import GetAtomVector, HybAtomTyper
import pandas as pd
import os, sys, gzip, pickle, glob, shutil
import subprocess as sp

In [3]:
df = pd.read_pickle('index_2019.pkl.gz')
df_selection = df[(df['type'] == 'Kd') & (df['lig_ok'] == True) & (df['select'] == True)]

### Create ligand and pocket

In [4]:
def get_pocket_atoms(protein_atoms, ligand_atoms):
    dist_mat = ligand_atoms.get_distance_matrix(protein_atoms)
    dist = {}
    for atom in protein_atoms:
        chainID = atom['chainID']
        resSeq = atom['resSeq']
        _one_atom = PDBAtoms()
        _one_atom.append(atom)
        d = _one_atom.get_distance_matrix(ligand_atoms)[0].min()
        if (chainID, resSeq) not in dist:
            dist[chainID, resSeq] = [d]
        else:
            dist[chainID, resSeq].append(d)
    for k in dist:
        dist[k] = sorted(dist[k])
    dist = sorted(dist.items(), key=lambda x: x[1][0])
    resSeqs = []
    atom_count = 0
    for resSeq, ds in dist:
        next_count = len(ds)
        if 150 < atom_count + next_count:
            break
        atom_count += next_count
        resSeqs.append(resSeq)
    resSeqs = set(resSeqs)

    pocket_atoms = PDBAtoms()
    for atom in protein_atoms:
        chainID = atom['chainID']
        resSeq = atom['resSeq']
        if (chainID, resSeq) in resSeqs:
            pocket_atoms.append(atom)

    return pocket_atoms

def check_ligand_atoms(ligand_mol, atomic_nums):
    d = set([atom.GetAtomicNum() for atom in ligand_mol.GetAtoms()]) - set(atomic_nums)
    return len(d) == 0

In [5]:
ligexpo = LigandExpo()

count = 0
for i in df_selection.index:
    r = df_selection.loc[i]

    pdb_code = r['pdb']
    ligand_name = r['lig']
    year = r['year'].item()
    value = r['pval'].item()

    if os.path.exists(f'work/{pdb_code}/{pdb_code}_pocket.pdb'):
        continue

    pdb_atoms = PDBAtoms(f'pdb/{pdb_code}.pdb.gz', removeWater=False, removeHet=False)
    ligand_atoms = pdb_atoms.get_ligand(ligand_name)

    if len(ligand_atoms) == 0:
        continue

    if 60 < len(ligand_atoms):
        continue

    smi = ligexpo[ligand_name]

    try:
        ligand_mol = ligexpo.assign(ligand_atoms, ligand_name)
    except:
        continue

    if not check_ligand_atoms(ligand_mol, [6, 7, 8, 9, 15, 16, 17, 35]):
        continue

    protein_atoms = pdb_atoms.get_interacting_chains(ligand_atoms)

    pocket_atoms = get_pocket_atoms(protein_atoms, ligand_atoms)

    count += 1
    print(count, pdb_code, ligand_name, len(ligand_atoms), len(pocket_atoms), len(protein_atoms))

    os.makedirs(f'work/{pdb_code}', exist_ok=True)
    open(f'work/{pdb_code}/{pdb_code}_pocket.pdb', 'wt').write(str(pocket_atoms))

    sdwriter = Chem.SDWriter(f'work/{pdb_code}/{ligand_name}.sdf')
    sdwriter.write(ligand_mol)
    sdwriter.close()
    gzip.open(f'work/{pdb_code}/{pdb_code}_apo.pdb.gz', 'wt').write(str(protein_atoms))
    open(f'work/{pdb_code}/year', 'wt').write(str(year))
    open(f'work/{pdb_code}/value', 'wt').write(str(value))    

RDKit ERROR: [23:24:30] Explicit valence for atom # 6 N, 6, is greater than permitted
RDKit ERROR: [23:24:30] Explicit valence for atom # 6 N, 6, is greater than permitted
RDKit ERROR: [23:24:30] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
RDKit ERROR: 
RDKit ERROR: [23:24:30] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
RDKit ERROR: 
RDKit ERROR: [23:24:30] Explicit valence for atom # 6 N, 4, is greater than permitted
RDKit ERROR: [23:24:32] Explicit valence for atom # 7 N, 5, is greater than permitted
RDKit ERROR: [23:24:32] Explicit valence for atom # 7 N, 5, is greater than permitted
RDKit ERROR: [23:24:33] Explicit valence for atom # 32 B, 4, is greater than permitted
RDKit WARNING: [23:24:30] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [23:24:30] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [23:24:30] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [23:24:31] WARNING: More than one 

In [ ]:
# LIGAND_THRES = 60
# POCKET_THRES = 240
# DISTANCE_THRES = 4.0
# METHOD = 'inclusive'
# 
# ligexpo = LigandExpo()
# 
# count = 0
# for i in df_selection.index:
#     r = df_selection.loc[i]
# 
#     pdb_code = r['pdb']
#     ligand_name = r['lig']
#     year = r['year'].item()
#     value = r['pval'].item()
# 
#     if os.path.exists(f'work/{pdb_code}/{pdb_code}_pocket.pdb'):
#         continue
# 
#     pdb_atoms = PDBAtoms(f'pdb/{pdb_code}.pdb.gz', removeWater=False, removeHet=False)
#     ligand_atoms = pdb_atoms.get_ligand(ligand_name)
# 
#     if len(ligand_atoms) == 0:
#         continue
#     if 60 < len(ligand_atoms):
#         continue
#     smi = ligexpo[ligand_name]
#     try:
#         ligand_mol = ligexpo.assign(ligand_atoms, ligand_name)
#     except:
#         continue
# 
#     ans = [atom.GetAtomicNum() for atom in ligand_mol.GetAtoms()]
#     ligand_ok = True
#     for a in ans:
#         if a not in [6,7,8,9,15,16,17,35]:
#             ligand_ok = False
#             break
#     if not ligand_ok:
#         continue
# 
#     protein_atoms = pdb_atoms.get_interacting_chains(ligand_atoms)
#     pocket_atoms = protein_atoms.get_pocket(ligand_atoms, thres=DISTANCE_THRES, method=METHOD)
#     if 240 < len(pocket_atoms):
#         continue
# 
#     count += 1
#     print(count, pdb_code, ligand_name, len(ligand_atoms), len(pocket_atoms), len(protein_atoms))
# 
#     os.makedirs(f'work/{pdb_code}', exist_ok=True)
#     open(f'work/{pdb_code}/{pdb_code}_pocket.pdb', 'wt').write(str(pocket_atoms))
# 
#     sdwriter = Chem.SDWriter(f'work/{pdb_code}/{ligand_name}.sdf')
#     sdwriter.write(ligand_mol)
#     sdwriter.close()
#     gzip.open(f'work/{pdb_code}/{pdb_code}_apo.pdb.gz', 'wt').write(str(protein_atoms))
#     open(f'work/{pdb_code}/year', 'wt').write(str(year))
#     open(f'work/{pdb_code}/value', 'wt').write(str(value))

### Create matrices

In [ ]:
todo = []
for root, dirs, files in os.walk('work'):
    ligand_name = None
    pdb_code = None
    for f in files:
        if f.endswith('.sdf') and len(f) == 7:
            ligand_name = f[:3]
            pdb_code = os.path.basename(root)
            break
    if not pdb_code or not ligand_name:
        continue
    test = f'{root}/{pdb_code}_{ligand_name}_data.pkl.gz'
    if os.path.exists(test):
        continue
    todo.append((pdb_code, ligand_name))
len(todo)

In [ ]:
LIGAND_THRES = 60
POCKET_THRES = 240
DISTANCE_THRES = 4.0
METHOD = 'inclusive'

ng_count = 0
count = 0
for pdb_code, ligand_name in todo:
    count += 1
    pdb_fname = f'pdb/{pdb_code}.pdb.gz'
    pdb_atoms = PDBAtoms(pdb_fname, removeWater=True)
    ligand_atoms = pdb_atoms.get_ligand(ligand_name)
    pocket_atoms = pdb_atoms.get_interacting_chains(ligand_atoms).get_pocket(ligand_atoms, thres=DISTANCE_THRES, method=METHOD)
    print(count, pdb_code, len(pocket_atoms), ligand_name)
    mat = GetTopologicalMatrix(ligand_atoms, pocket_atoms, ligand_thres=60, pocket_thres=240)
    ligand_sdf = f'work/{pdb_code}/{ligand_name}.sdf'
    for ligand_mol in Chem.SDMolSupplier(ligand_sdf):
        break
    if not ligand_mol:
        ng_count += 1
        print('Error', count, pdb_code, 'ligand_error', file=sys.stderr)
        continue
    try:
        types_vec = GetAtomVector(ligand_mol, pocket_atoms, atomtyper=HybAtomTyper, ligand_thres=LIGAND_THRES, pocket_thres=POCKET_THRES)
    except:
        ng_count += 1
        print('Error', count, pdb_code, 'vector_error', file=sys.stderr)
        continue
    data = dict(A=types_vec, B=mat)
    pickle.dump(data, gzip.open(f'work/{pdb_code}/{pdb_code}_{ligand_name}_data.pkl.gz', 'wb'), protocol=4)
print(ng_count, count)

<hr/>

In [ ]:
len(glob.glob('work/*/*_data.pkl.gz'))

In [ ]:
len(glob.glob('work/*/*_pair.pkl.gz'))

<hr/>

In [ ]:
todo = []
for root, dirs, files in os.walk('work'):
    ligand_name = None
    pdb_code = None
    for f in files:
        if f.endswith('.sdf') and len(f) == 7:
            ligand_name = f[:3]
            pdb_code = os.path.basename(root)
            break
    if not pdb_code or not ligand_name:
        continue
    test1 = f'{root}/{ligand_name}.sdf'
    test2 = f'{root}/{pdb_code}_pocket.pdb'
    if not (os.path.exists(test1) and os.path.exists(test2)):
        continue
    todo.append((pdb_code, ligand_name))
len(todo)

In [ ]:
os.makedirs('2018', exist_ok=True)
os.makedirs('select', exist_ok=True)

df = pd.read_pickle('index_2019.pkl.gz')

counts = {'select': 0, '2018': 0}

for pdb_code, ligand_name in todo:
    r = df[df['pdb'] == pdb_code]
    year = r['year'].item()
    ligand_name = r['lig'].item()
    select = r['select'].item()

    if year == 2018:
        counts['2018'] += 1
        dest = f'2018/{pdb_code}'
    else:
        counts['select'] += 1
        dest = f'select/{pdb_code}'
    
    os.makedirs(dest, exist_ok=True)

    ligand_fname = f'work/{pdb_code}/{ligand_name}.sdf'
    shutil.copy(ligand_fname, dest)
    pocket_fname = f'work/{pdb_code}/{pdb_code}_pocket.pdb'
    shutil.copy(pocket_fname, dest)
    value_fname = f'work/{pdb_code}/value'
    shutil.copy(value_fname, dest)

sp.call(f'tar jcvf cbidata_gnn_dti.tar.bz2 2018/ select/', shell=True)

shutil.rmtree('2018')
shutil.rmtree('select')
counts

<hr/>

In [ ]:
of_select = open('keys_select', 'wt')
of_2018 = open('keys_2018', 'wt')
for pdb_code, ligand_name in sorted(todo):
    value = float(open(f'work/{pdb_code}/value').read())
    year = int(open(f'work/{pdb_code}/year').read())
    ofs = of_2018 if year == 2018 else of_select
    print(pdb_code, ligand_name, year, value, sep='\t', file=ofs)
of_select.close()
of_2018.close()